# 第 2 章：数值优化（Optimization for Pruning, Quantization & Distillation）

本章目标：理解剪枝、量化、蒸馏等操作背后的“优化问题”视角。  
你不需要成为优化理论专家，但需要：

- 看懂常见损失函数与正则项的形式
- 理解 L0/L1 稀疏化、量化参数优化的大致思路
- 知道多目标优化（精度 vs 延迟）的典型写法

---

## 2.1 剪枝（Pruning）的优化视角

### 2.1.1 基本目标

给定训练好的权重 $W$、损失函数 $L(W)$，剪枝希望：

- 大量元素变为 0（稀疏）
- 损失/精度变化尽量小

可以形式化为：

$$
\min_{\hat{W}} L(\hat{W}) + \lambda \|\hat{W}\|_0
$$


其中：
- 这里 $L$ 是模型的损失函数（训练时用的目标函数），在权重为 $\hat{W}$ 时的损失值
- $\|\hat{W}\|_0$：非零元素个数（稀疏性度量）
- $\lambda$：权衡“精度 vs 稀疏度”的超参数

这个问题一般是 NP-hard，常见近似方式有：

1. 用 $L_1$ 范数替代 $L_0$：

$$
\min_{\hat{W}} L(\hat{W}) + \lambda \|\hat{W}\|_1
$$


2. 先正常训练，再做基于某种 saliency 的后处理剪枝：
   - 按权重绝对值大小剪掉小值
   - 按 Hessian 近似（如 OBS/OBD）计算敏感度

---

### 2.1.2  Hessian矩阵 & saliency score
后训练剪枝（post-training pruning）核心是

- 这条边（某个 weight）删掉会不会伤害 loss？

- 哪个通道更重要？

- 哪层该多剪？哪层不能乱动？j z

🔷 1. 什么是 Hessian？
Hessian 是 loss 对参数的二阶导数矩阵：
$$H = \nabla^2_{W} L(W)$$
更精确地：
- 如果模型参数是$$W \in \mathbb{R}^n$$， 则 Hessian 是：$$H \in \mathbb{R}^{n \times n}$$
其中每个元素： $$H_{ij} = \frac{\partial^2 L}{\partial W_i \partial W_j}$$
代表 loss 对两个 weight 的二阶相互影响。

🔷 2. 为什么 Hessian 与剪枝有关？

问题：
如果把某个 weight从 $w_i$->0, loss 会变多少？
如果你直接看$|w_i|$,只能知道它“大小”，不知道：
- 这个权重对 loss 的敏感度如何

- 换句话说，它是不是“关键路径”上的重要参数？

简单绝对值是**粗糙代理**。Hessian 则衡量了：**改变一个权重，会让 loss 增大多少**, 更精确。

🔥 3. 用二阶泰勒展开估计“剪掉某个 weight 会带来多少损失增加”？
假设我们要把第$i$个 weight从当前$w_i$设置成 0：
$$\Delta w_i = -w_i$$
loss 的变化（用二阶泰勒展开）：
$$\Delta L \approx 
\frac{\partial L}{\partial w_i}\Delta w_i + \frac{1}{2} H_{ii} (\Delta w_i)^2$$
但：
- 在训练完的点, 梯度$\partial L/\partial w_i \approx 0$（因为训练收敛）
- 所以剩下：$$\Delta L \approx \frac{1}{2} H_{ii} w_i^2$$

至此 **saliency score** = $H_{ii} w_i^2$
这比单看$|w_i|$更有意义

🔷 直觉的解释
- 泰勒展开说：
“在一个点附近，函数的变化 ≈ 一阶（斜率） + 二阶（弯曲程度）。”

- 在已经训练好的点：
一阶导（斜率）接近 0，所以变化主要靠二阶导决定。

- 对于剪枝：
你把$w_i$改成 0，相当于在参数空间里沿着第 i 个坐标方向，走了一个步长 $-w_i$
损失增加 ≈ 弯曲程度 × 你走的距离的平方。
	​
- “弯曲程度”在这个方向就是 Hessian 的对角线元素 $H_{ii}$
 - 如果这个方向上弯得很厉害（H 大），说明对这个参数很敏感，不能乱剪。
 - 如果这个方向几乎是平的（H 小），即使 $w_i$不为零，也可以大胆剪

所以：
- 只看$|w_i|$ → 只是看这个参数“大不大”。
- 看$H_{ii} w_i^2$ → 看“删掉它造成 loss 上升有多大”，这是更合理的“重要性”度量。

🔷 4. OBS（Optimal Brain Surgeon）与 OBD（Optimal Brain Damage）
这是 1990 年代的经典剪枝方法：

🟦 OBD（Optimal Brain Damage）

它使用对角 Hessian：
$$\text{saliency}_i = \frac{1}{2} H_{ii} w_i^2$$
直觉：
* 如果 $H_{ii}$很大 → 该weight 非常敏感
* 就算它很小，也不能轻易删
* 如果 $H_{ii}$ 很小 → 这个 weight 不重要
* 可以安全剪掉

缺点：忽略 Hessian 的 off-diagonal（跨 weight 相关性）。

🟩 OBS（Optimal Brain Surgeon）
OBS 更精确：
$$\text{saliency}_i = 
\frac{1}{2} \frac{w_i^2}{(H^{-1})_{ii}}$$

OBS 使用逆 Hessian，考虑所有 weight 之间的相互影响，理论上更准确：

- 如果 weight 能被其他参数补偿（Hessian inverse 反映相关性），它更适合删
- 但计算 Hessian inverse 太贵，现代很少用全量 OBS。

🔥 5. 工程做法（Pruning-aware methods）
真实工程中，我们不可能算真正的 Hessian（太大），但可以：

✔ 近似对角线：
- 用 Fisher 信息矩阵对角线
- 用 Hutchinson estimator
- 用梯度平方的 moving average , 类似 Adam 的 $v_t$ ，这就把 Hessian 估计变成可训练、可部署的东西。
此处
$H_{ii} \approx \mathbb{E}[g_i^2]$， 

$g_i = \frac{\partial L}{\partial w_i}$
这个梯度随着 batch 而变化，因为 loss 是基于 mini-batch 求出来的，所以通常用：
$$\mathbb{E}[g_i^2] = \text{over minibatches }$$
换句话说：

- 每个 batch 会算一次梯度 $g = \nabla L$
- 取它的平方 $g_i^2$
- 再对 batch 求期望（平均）

就得到了对角线 Hessian 的估计。


📌 为什么梯度平方可以近似 Hessian 对角线？
在深度学习中，尤其是使用 **Cross-Entropy loss + Softmax** 的模型，（其他的不一定对，比如MSE（平方误差回归））里：
$$H_{ii} \approx \mathbb{E}[g_i^2]$$
原因是：
1. Hessian = 二阶变化率；Fisher 信息 = 梯度方差
对数似然条件下，有数学定理（Cramer–Rao + Information Equality）：
$$\mathbb{E}[ \nabla^2 L ] = \mathbb{E}[ g g^\top ]$$
取对角线：
$$H_{ii} \approx \mathbb{E}[g_i^2]$$
换句话说：
**梯度的方差 = 二阶曲率（Hessian 对角线**

📌 工程上如何计算$E[g_i^2]$?
```code
hessian_diag[i] += grad[i]**2
count += 1
...
hessian_diag /= count
```
这就是 Hessian 对角线的无偏估计。

📌 直觉的解释
- 梯度是“损失对参数的一阶反应”

- 梯度平方的期望就是“损失在这个方向上的波动强度”

- 波动强说明“这个方向上 loss 很陡” → Hessian 大

- 波动弱说明“这个方向平缓” → Hessian 小
所以：
$H_{ii} \approx E[g_i^2]$
---

#### 2.1.2.1 二阶泰勒展开详细推导
现在参数是向量： $W = (w_1, w_2, \dots, w_n)^\top$
loss 是 $L(W)$
在点 𝑊 附近做二阶泰勒展开：
$$L(W + \Delta W)
\approx
L(W)
+ \nabla L(W)^\top \Delta W
+ \frac{1}{2} \Delta W^\top H \Delta W$$

这里：
- $\nabla L(W)$是梯度向量（长度 n）
- $H = \nabla^2 L(W)$ 是 Hessian 矩阵（n×n

现在 **我们只动第 i 个参数，把它变成 0**
也就是说：
- 原来：$w_i$
- 剪枝后：$w_i^{\text{new}} = 0$
所以：
$$\Delta W = W_{\text{new}} - W_{\text{old}}
= (0,\dots,0, -w_i, 0,\dots,0)^\top
= -w_i e_i$$

代入泰勒展开公式
$$\Delta L
= L(W + \Delta W) - L(W)
\approx \nabla L(W)^\top \Delta W + \frac{1}{2} \Delta W^\top H \Delta W$$

梯度向量：
$\begin{array}{c} \nabla L(W) = 
\begin{bmatrix}
\frac{\partial L}{\partial w_1} \\
\frac{\partial L}{\partial w_2} \\
\vdots \\
\frac{\partial L}{\partial w_n}
\end{bmatrix} \end{array}$


把 $\Delta W = -w_i e_i$代进去：
- 一阶项：
$$\nabla L(W)^\top \Delta W
= \nabla L(W)^\top (-w_i e_i)
= -w_i \frac{\partial L}{\partial w_i}$$

- 二阶项
$$\Delta W^\top H \Delta W
= (-w_i e_i)^\top H (-w_i e_i)
= w_i^2 \, e_i^\top H e_i$$
但：
$$e_i^\top H e_i = H_{ii}$$
所以：
$$\Delta W^\top H \Delta W = w_i^2 H_{ii}$$
于是：
$$\Delta L \approx -w_i \frac{\partial L}{\partial w_i} + \frac{1}{2} w_i^2 H_{ii}$$

**再次使用“在收敛点梯度 ≈ 0”**, 在训练好的参数$W^\star$附近：
$$\frac{\partial L}{\partial w_i}(W^\star) \approx 0$$
于是：
$$\Delta L \approx \frac{1}{2} H_{ii} w_i^2$$
这就是**OBD（Optimal Brain Damage）中的 saliency**: $\text{saliency}_i = \frac{1}{2} H_{ii} w_i^2$


#### 2.1.2.2 pytorch CrossEntropy 下的 Hessian 对角线 vs E[g²]

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd.functional import hessian

torch.manual_seed(0)

# ===========================
# 1. 构造一个小的 logistic 回归模型
#    y ∈ {0,1}, p = sigmoid(xW + b)
# ===========================

in_dim = 2

N = 500
X = torch.randn(N, in_dim)

# 真实参数（只用于生成数据）
true_theta = torch.tensor([2.0, -3.0, 0.5])  # [w1, w2, b]
true_W = true_theta[:in_dim].view(in_dim, 1) # [2,1]
true_b = true_theta[in_dim:].view(1)         # [1]

logits = X @ true_W + true_b            # [N,1]
probs = torch.sigmoid(logits)           # [N,1]
y = torch.bernoulli(probs).view(-1).long()  # 转成 int labels [N]

# 我们同样用一维参数向量 theta = [w1, w2, b] 来优化
theta = torch.randn(in_dim + 1, requires_grad=True)

def unpack_theta(th):
    W = th[:in_dim].view(in_dim, 1)   # [2,1]
    b = th[in_dim:].view(1)           # [1]
    return W, b

def loss_fn(theta, X, y):
    W, b = unpack_theta(theta)        # W: [2,1], b: [1]
    logits = X @ W + b                # [N,1]
    logits = logits.view(-1, 1)
    # Binary cross entropy with logits
    loss = F.binary_cross_entropy_with_logits(logits, y.float().view(-1, 1))
    return loss

# 简单训练一下，让 theta 接近“收敛点”
optim = torch.optim.SGD([theta], lr=0.1)

for step in range(300):
    optim.zero_grad()
    loss = loss_fn(theta, X, y)
    loss.backward()
    optim.step()

print("Trained loss:", loss_fn(theta, X, y).item())
print("Trained theta:", theta.detach().numpy())

# ==========================================
# 2. 计算“真·Hessian 对角线”：H_ii
# ==========================================

def loss_only_theta(th):
    return loss_fn(th, X, y)

H = hessian(loss_only_theta, theta)   # [P,P], P = in_dim+1=3
H_diag_true = torch.diag(H).detach()

print("\nTrue Hessian diagonal H_ii:")
print(H_diag_true)

# ==========================================
# 3. 用多 batch 的梯度平方平均 E[g^2] 近似 H_ii
# ==========================================

num_batches = 100
batch_size = 64

g2_accum = torch.zeros_like(theta)

for _ in range(num_batches):
    idx = torch.randint(0, N, (batch_size,))
    Xb = X[idx]
    yb = y[idx]

    loss_b = loss_fn(theta, Xb, yb)
    grad_theta = torch.autograd.grad(loss_b, theta, retain_graph=False, create_graph=False)[0]

    g2_accum += grad_theta.detach() ** 2

H_diag_est = g2_accum / num_batches

print("\nEstimated Hessian diagonal via E[g^2]:")
print(H_diag_est)

# ==========================================
# 4. 对比两者
# ==========================================

abs_diff = (H_diag_true - H_diag_est).abs()
rel_diff = abs_diff / (H_diag_true.abs() + 1e-8)

print("\nAbsolute diff |H_ii - E[g_i^2]|:")
print(abs_diff)

print("\nRelative diff |H_ii - E[g_i^2]| / |H_ii|:")
print(rel_diff)

Trained loss: 0.33923059701919556
Trained theta: [ 1.3182789 -1.9975102  0.4114626]

True Hessian diagonal H_ii:
tensor([0.0965, 0.0759, 0.1324])

Estimated Hessian diagonal via E[g^2]:
tensor([0.0015, 0.0012, 0.0014])

Absolute diff |H_ii - E[g_i^2]|:
tensor([0.0949, 0.0747, 0.1310])

Relative diff |H_ii - E[g_i^2]| / |H_ii|:
tensor([0.9839, 0.9840, 0.9891])


### 2.1.3 补：Fisher 信息矩阵
Fisher 信息矩阵（Fisher Information Matrix, FIM） 衡量的是：

模型对参数变化的敏感度

或

用参数去解释数据时，不确定性有多小

“信息越大 → 参数越重要，不能动”
“信息越小 → 参数不重要，可以剪 / 量化得更激进”

---
1. Fisher 信息矩阵的正式定义
给定模型的对数似然（log-likelihood）：$\log p_\theta(x)$

Fisher 信息矩阵定义为：
$$F(\theta) = \mathbb{E}\left[ \left( \nabla_\theta \log p_\theta(x) \right)
       \left( \nabla_\theta \log p_\theta(x) \right)^\top \right]$$

也就是说：

- 先对 log-likelihood 求梯度（score function）

- 再求它与自身的外积

- 再在数据分布上取期望

---
2. 深度学习里更常用的表达
当我们用 CrossEntropy Loss 时：
$$L(\theta) = -\log p_\theta(y|x)$$
梯度就是：
$$g = \nabla_\theta L = - \nabla_\theta \log p_\theta(y|x)$$
于是 Fisher 信息矩阵等价于：
$$F(\theta) = \mathbb{E}[g\, g^\top]$$

对角线就是：
$$F_{ii} = \mathbb{E}[g_i^2]$$
也就是
$$H_{ii} \approx \mathbb{E}[g_i^2]$$

---
3. 为什么 Fisher 信息矩阵在深度学习中很重要？
Fisher 信息告诉我们：

如果参数 θ 发生微小变化，模型预测会有多大波动？

换句话说：
- $F_{ii}$ 大 → 参数非常“敏感” → 很重要 → 不宜剪
- $F_{ii}$小 → 参数基本不起作用 → 可以剪 / 可以低 bit 量化 

所以：

✔ Fisher 信息可用于剪枝（pruning saliency）

✔ 可用于量化（quantization sensitivity）

✔ 可用于确定哪些层的 rank 要下调（low-rank SVD）

✔ 可用于 LayerNorm / Whitening 的稳定性分析


### 2.1.4 结构化剪枝（channel/head pruning）
结构化剪枝关注“整个通道/整个 head/整个 filter 是否重要”，而不是单个 weight。
可以把通道剪枝写成：

$$
\min_{\theta, m} \ L(\theta \odot m) + \lambda \|m\|_0
$$


- $\theta$：原始参数
- $m$：mask，按通道/块为单位取 0 或 1
- $\odot$：逐元素或逐通道乘法

通常会：

1. 训练一个带有可微近似 mask 的模型（如用 sigmoid/Concrete distribution）
2. 在训练过程中逐渐压缩某些通道的权重
3. 最后将接近 0 的通道硬剪掉

> 本质仍然是“带稀疏约束的优化问题”，只是作用对象从单个元素提升到了“结构化块”。

---


### 2.1.5 各种剪枝方法对比
| 方法                           | 使用的指标                         | 指标意义                     | 是否衡量重要性？        | 是否科学？ |
| ------------------------------ | ----------------------------------- | ----------------------------- | ------------------------- | ---------- |
| **L1 剪枝**                    | $\lvert w_i \rvert$                | 是否接近 0（自然稀疏）         | ❌ 否                     | 低         |
| **Hessian 剪枝（OBD/OBS）**    | $H_{ii} \, w_i^2$                  | 删掉该权重造成的 loss 增加量  | ✔ 是（数学严格）         | 高         |
| **Fisher 剪枝**                | \mathbb{E}[g_i^2] \, w_i^2$        | 用 Fisher 近似 Hessian       | ✔ 是                     | 高         |
| **Magnitude 剪枝（structured）** | 通道 $\ell_1$ / $\ell_2$ 范数       | 通道能量大小                  | 部分                      | 中         |



## 2.2 量化（Quantization）的优化视角

量化本质不是简单地“把浮点变成 int8/int4”，
而是：

**在 bit-budget（位宽预算）受限的约束下，求一个逼近原模型的最优离散表示，使得 inference 的误差最小。**

数学形式上就是：
$$\min_{s,z, Q(\cdot)} \; L\big(f_{Q,s,z}(x), f(x)\big)$$
其中：
- $Q(\cdot)$量化映射（round, clip）
- $s$: scale （步长）
- $z$: zero point（非对称量化的偏置）
- $f_{Q,s,z}$: 量化后的模型
- $f$: 模型

---

#1 量化的三个核心优化目标
📌 目标 1：最小化量化误差
最常见：
$$\min_{s,z} \sum_i (w_i - \hat{w}_i)^2$$
其中：
$$\hat{w}_i = s (Q(w_i/s) - z)$$
这是 最小二乘意义下 的最优 scale / zero point。


📌 目标 2：满足 bit-budget（约束优化）
$q_i \in \{-Q, ..., Q\}, \quad Q = 2^{b-1} - 1$
bit 越少 → 量化噪声越大 → 推理误差越高
本质是：**在 bit=4/8 的约束下，把误差压到最低。**

📌 目标 3：维持推理解码稳定性（梯度无关）
- runtime inference 中最重要：

- 不爆值

- 不溢出

- 不积累数值漂移

- softmax 不崩

- LayerNorm 保持稳定
数学上，这要求：

$$\|f_{Q}(x) - f(x)\| \text{ 小且可控}$$
特别是在 Transformer 中，量化误差会在注意力路径累计，需要针对不同层设计不同优化策略



### 2.2.1 简单的 uniform quantization - 优化scale

考虑对权重集合 $\{w_i\}$ 做对称均匀量化，bit 宽为 b：

- 量化级别：$q_i \in \{-Q,\dots,Q\}$，其中 $Q = 2^{b-1}-1$
- 缩放因子（scale）：$s > 0$
- 量化/反量化过程：

$$
q_i = \text{round}(w_i / s), \quad
  \hat{w}_i = s \cdot q_i
$$


典型目标是最小化量化误差：

$$
\min_s \sum_i (w_i - \hat{w}_i)^2
$$


这实际上是一个一维凸优化问题。可以证明：

- 对称量化（zero-point = 0）
- 量化区间固定在$[-Q, Q]$
则最优 scale 解为：$$s^* = \frac{\max |w_i|}{Q}$$

工程上常见 heuristics：

- 直接取：

$$
s = \frac{\max_i |w_i|}{Q}
$$


- 或取某个分位数（如 99.9%）替代 max，防止 outlier 过大：

$$
s = \frac{\text{quantile}_{p}(|w_i|)}{Q}
$$


**【要点】**  
你需要知道：

- 量化参数（scale/zero-point）可以通过最小二乘意义下的优化求得
- 工程实现中用 histogram + 搜索 / heuristics 做近似求解： 原因真实权重/激活 不是均匀分布，很多 heavy-tail（长尾）的。所以更好的优化方法是**优化分布逼近：KL 最小化**
$$s^* = \arg\min_s \text{KL}(P(w) \| P(\hat{w}))$$
所以很多工具（TensorRT、TF-Lite、MKLDNN etc）都用：

 - histogram 分布

 - quantization reconstruction

 - KL divergence

来选最优 clip range。


---

### 2.2.2 LSQ（Learned Step Size Quantization）思想

更进一步，可以把 scale $s$ 当成可学习参数，让反向传播直接优化：

- 定义一个“伪量化”算子（用 STE Straight-Through Estimator 逼近梯度）
- 在训练/微调时 joint optimize $W$ 和 $s$
把 quantizer 写成（带 straight-through 梯度）：
$$\hat{w} = \mathrm{clip}\Big(\mathrm{round}(\frac{w}{s})\Big) \cdot s$$
然后把 s 当成模型参数一起训练：
$$\min_{s,W} L(\hat{W}(s), y)$$
这就从数学上变成：

- 联合优化（joint optimization）

- 带不可导算子的近似梯度（STE）

- 参数化的约束优化

这种方法使得量化参数针对当前任务/数据集自适应地收敛到较好的值，从而提升低 bit 量化的精度。

#### 2.2.2.1 STE Straight-Through Estimator
当量化、取整、符号函数等 不可导（梯度为 0 或不存在）时，
无法直接做反向传播。

STE 的核心思想是：**前向使用真实的非可导算子（如 round、sign），反向时“假装它是恒等映射”，把梯度直接传过去。**
数学写法：
前向：$$\hat{w} = \text{round}(w)$$
反向：$$\frac{\partial \hat{w}}{\partial w} \approx 1$$
或者根据不同变体：
$$\begin{array}{c} \frac{\partial \hat{w}}{\partial w} = 
\begin{cases}
1, & |w|<1 \\
0, & \text{else}
\end{cases} \end{array}$$

---
🔷 为什么叫 "Straight-Through"？
因为在反向传播时，梯度不被阻断，而是“直接穿过（go straight through）”不可导算子。
```block
   w ---->[ round ]----> \hat{w}
           (non-diff)

   backward: gradient just passes straight through

```
- 前向用离散值, $\hat{w} = s \cdot \text{round}(w/s)$
- 反向把 round 当成 identity. $\frac{\partial \hat{w}}{\partial w} \approx 1$
因此量化模型可以继续训练（QAT）、学习最优 scale（LSQ）。

🔷 STE 特别适用于：

- QAT（Quantization-Aware Training）

- LSQ（Learned Step Size Quantization）

- 二值化网络（BNN）

- hard-sigmoid/硬门控的梯度近似

- 任何涉及 “不可导函数 + 想训练它” 的场景

---


## 2.3 知识蒸馏（Distillation）的优化视角

### 2.3.1 核心思想：蒸馏是一个联合最优化问题

Teacher（大模型）向 Student（小模型）传递软知识（soft knowledge）。

- Teacher logits：  
  $$ z^T $$

- Student logits：  
  $$ z^S $$

加入温度 $ T $ 的 softmax：

$$
p^T_i = \frac{e^{z_i^T / T}}{\sum_j e^{z_j^T / T}}, 
\qquad
p^S_i = \frac{e^{z_i^S / T}}{\sum_j e^{z_j^S / T}}
$$

Student 的目标是最小化以下联合损失：

$$
L = \alpha L_{\text{CE}} + (1 - \alpha)L_{\text{KL}}
$$

---

### 2.3.2 Hard Loss（交叉熵 CE）

学生对真实标签的交叉熵损失：

$$
L_{\text{CE}} = -\sum_i y_i \log p^S_i
$$

- $y$：one-hot 或 label smoothing 处理后的标签  
- 这一项保证 Student 至少能对“标准答案”做对分类

---

### 2.3.3 Soft Loss（KL 蒸馏损失）

学生模仿 Teacher 的输出分布：

$$
L_{\text{KL}} = T^2 \sum_i p^T_i \log \frac{p^T_i}{p^S_i}
$$

- 使用 $ T^2 $ 来抵消温度对梯度的缩放  
- Teacher 的 soft labels 提供“暗知识”（dark knowledge）  
- 不仅告诉 Student “谁是对的”，还告诉它 “谁跟谁更像”

联合损失为：

$$
L = \alpha L_{\text{CE}} + (1-\alpha)L_{\text{KL}}
$$

其中：

- $ \alpha $：控制对真实标签（hard label）和 Teacher（soft label）的信任度  
- 大模型越强，一般可以让 $ \alpha $ 稍微小一点（更依赖 Teacher）

---

### 2.3.4 KL 蒸馏梯度的直观推导

KL 部分对 Student logits 的梯度：

$$
\frac{\partial L_{\text{KL}}}{\partial z_k^S} 
= p_k^S - p_k^T
$$

直观解释：

- 学生输出比 Teacher 大（$ p_k^S > p_k^T $） → 梯度为正 → 下降时往下拉  
- 学生输出比 Teacher 小（$ p_k^S < p_k^T $） → 梯度为负 → 下降时往上推  

也就是说：

> Student 调整自己的输出概率，使其逐步逼近 Teacher 的输出概率分布。

与交叉熵 CE 的梯度对比：

$$
\frac{\partial L_{\text{CE}}}{\partial z_k^S} = p_k^S - y_k
$$

- 对 CE 来说，target 是 one-hot 的 $y_k$
- 对 KL 蒸馏来说，target 是 soft 的 $p_k^T$

因此：

> 蒸馏 = 用 Teacher 的软分布 $p^T$ 替代硬分布 $y$ 来“监督” Student。

---
#### 2.3.4.x KL梯度推导

Student 概率（softmax）： 
$$p^S_i = \frac{e^{z^S_i}}{\sum_j e^{z^S_j}}$$

KL 蒸馏损失（忽略前面的 $T^2$）：
$$L_{\mathrm{KL}} = \sum_i p^T_i \log \frac{p^T_i}{p^S_i}
               = \sum_i p^T_i \big(\log p^T_i - \log p^S_i\big)$$

其中：
- $\log p^T_i$部分与 Student 无关（常数）
- 真正和 student 有关的是这一项
$$L' = - \sum_i p^T_i \log p^S_i$$
这其实就是以 **$p^T$为 label 的交叉熵**
$$L' = H(p^T, p^S)$$

所以本质就是： **KL 的梯度 = 交叉熵 w.r.t. logits 的梯度。**

对 logits 求导：$\partial L' / \partial z_k^S$
$$L' = - \sum_i p^T_i \log p^S_i$$
- 第一步：对 $p^S_i$ 求偏导（链式法则）

$$\frac{\partial L'}{\partial z_k^S} = \sum_i \frac{\partial L'}{\partial p^S_i} \cdot \frac{\partial p^S_i}{\partial z_k^S}$$
先算第一部分：
$$\frac{\partial L'}{\partial p^S_i} = - p^T_i \cdot \frac{1}{p^S_i}$$

第二步：softmax 的导数

对 softmax：
$$p^S_i = \frac{e^{z^S_i}}{\sum_j e^{z^S_j}}$$
而
$$\begin{array}{c} \frac{\partial p^S_i}{\partial z_k^S}
=\begin{cases}
p^S_i (1 - p^S_i), & i = k \\
- p^S_i p^S_k, & i \ne k
\end{cases} \end{array}$$


- 第二步：把两部分乘起来并求和
$$\frac{\partial L'}{\partial z_k^S}
= \sum_i \left(- \frac{p^T_i}{p^S_i}\right) \cdot \frac{\partial p^S_i}{\partial z_k^S}$$
- i = k：
$$\left(- \frac{p^T_k}{p^S_k}\right) \cdot p^S_k(1 - p^S_k)
= - p^T_k (1 - p^S_k)
= -p^T_k + p^T_k p^S_k$$
- i ≠ k：
$$\sum_{i \ne k} \left(- \frac{p^T_i}{p^S_i}\right) \cdot (-p^S_i p^S_k)
= \sum_{i \ne k} p^T_i p^S_k
= p^S_k \sum_{i \ne k} p^T_i$$

==>
$$\frac{\partial L'}{\partial z_k^S}
= \big(-p^T_k + p^T_k p^S_k\big) + p^S_k \sum_{i \ne k} p^T_i$$

注意到 $\sum_{i \ne k} p^T_i = 1 - p^T_k$

所以：
$$\frac{\partial L'}{\partial z_k^S}
= -p^T_k + p^T_k p^S_k + p^S_k (1 - p^T_k)
= -p^T_k + p^T_k p^S_k + p^S_k - p^S_k p^T_k
= p^S_k - p^T_k$$

$$\boxed{
\frac{\partial L_{\mathrm{KL}}}{\partial z_k^S} 
= p^S_k - p^T_k
}$$

--- 

### 2.3.5 Temperature $ T $ 的作用（数学解释）

带温度的 softmax：

$$
p_i = \frac{e^{z_i/T}}{\sum_j e^{z_j/T}}
$$

- $ T > 1 $：分布更平滑（各类概率更均匀，类间关系更明显）  
- $ T < 1 $：分布更尖锐（更接近 argmax）  

蒸馏一般使用 $T = 2 ～ 4$ ，原因是：

- 高温使 softmax 输出对 logits 的变化更加“敏感且平滑”  
- KL-loss 对所有类别都产生梯度，而不仅仅是 top-1  
- Student 能“看见” Teacher 对所有类别的相对偏好，学到更丰富的结构信息

---

### 2.3.6 实际工程中的蒸馏流程（Teacher → Student）

一个典型的蒸馏 pipeline：

1. **训练 Teacher（大模型）**  
   - 用常规 FP32 或混合精度训练  
   - 得到性能较好的大模型 $f_T$

2. **准备 Teacher 输出**  
   - 对训练集或额外的蒸馏数据集跑 Teacher  
   - 保存每个样本的 Teacher logits $z^T$ 或 soft probabilities $p^T$

3. **构建 Student（小模型）**  
   - 更小的宽度/深度  
   - 更易量化/剪枝的结构（例如更窄的 MLP、部分 head 剪枝）

4. **联合训练 Student**  
   - 前向：Student 输出 logits $z^S$、概率 $p^S$  
   - 计算：
     - hard loss：$ L_{\text{CE}}(y, p^S) $
     - soft loss：$ L_{\text{KL}}(p^T \| p^S) $
   - 组合：
     $$L = \alpha L_{\text{CE}} + (1-\alpha)L_{\text{KL}}$$
   - 反向传播，更新 Student 参数

5. **（可选）增加特征蒸馏（Feature Distillation）**  
   - 强制 Student 的中间层特征接近 Teacher：
     $$
     L_{\text{feat}} = \| h^S - P(h^T) \|_2^2
     $$
   - 其中 $P(\cdot)$ 是投影层，用于对齐维度。

6. **（可选）增加注意力蒸馏（Attention Distillation）**  
   - 对 Transformer：
     $$
     L_{\text{attn}} = \| A^S - A^T \|_2^2
     $$
   - 其中 $A$ 是 Attention map（如 softmax(QK^\top / \sqrt{d_k})）

7. **得到最终的 Student**  
   - 更轻量、更快、更适合量化与剪枝  
   - 常用于边缘设备部署（车端、移动端、robot 等）

---

### 2.3.7 与 Runtime Inference 场景的关系

在部署场景中，你可以：

- 先训练一个较大的 Teacher（可能在云端/数据中心）  
- 然后设计一个为 **车端/ECU/NPU 优化** 的 Student：
  - 更小的宽度/深度  
  - 更友好的张量形状和 kernel 映射  
  - 便于 int8 / int4 量化、低秩分解、结构化剪枝

然后：

1. 用知识蒸馏保证 Student 在精度上尽量逼近 Teacher  
2. 再对 Student 做：
   - 量化（PTQ/QAT）
   - 剪枝（结构化通道/头剪枝）
   - low-rank/SVD 分解
3. 最终导出一个 **“蒸馏 + 压缩 + 量化” 一体化的部署版本**

> 换句话说：  
> **蒸馏不是“压模型”，而是“把大模型的函数空间投影到小模型的函数子空间”。**  
> 这就是它在数学和工程上的核心意义。

---



## 2.4 多目标优化：精度 vs 延迟 vs 内存

在做部署时，你不会只关心 loss，还会关心：

- latency（推理延迟）
- memory（显存 / DRAM 占用）
- throughput（QPS）

可以将这些纳入优化目标：

$$
\min_{\theta} \quad \alpha \cdot \text{Error}(\theta)
+ \beta \cdot \text{Latency}(\theta)
+ \gamma \cdot \text{Memory}(\theta)
$$


其中：

- $\theta$：模型结构 + 量化配置 + 剪枝策略 等
- $\text{Latency}(\theta)$：通过 profile 或 analytical model 估计
- $\text{Memory}(\theta)$：由参数量、activation、KV cache 决定

工程实现中常见简化：

- 固定某个 latency/memory 上限作为约束：

$$
\min_{\theta} \text{Error}(\theta)
  \quad \text{s.t.} \quad \text{Latency}(\theta) \le L_{\max}, \
  \text{Memory}(\theta) \le M_{\max}
$$


- 或把 latency/memory 转换为正则项加入 loss

> 重要的是：你要能把“部署需求”翻译成数学上的“目标 + 约束”，这样才能用优化工具系统地设计算法。

---

## 2.5 本章小结

本章重点：

1. 剪枝可以看作带 $L_0/L_1$ 稀疏约束的优化问题
2. 量化可以通过最小化量化误差的优化问题确定 scale/zero-point，进一步可通过训练 joint optimize
3. 蒸馏是一个联合最小化 CE 和 KL 的多目标优化问题
4. 部署时的精度、延迟、内存可以统一本为多目标/带约束的优化问题

> 在后续章节中，这些优化目标会与近似理论、概率统计、硬件模型结合在一起，形成一套完整的推理加速思维。

In [17]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Use pretrained ResNet-18 as teacher
teacher = resnet18(weights="IMAGENET1K_V1")
# fine tune last layer for CIFAR-10
teacher.fc = nn.Linear(512, 10)
teacher = teacher.to(device)
opt_teacher = torch.optim.Adam(teacher.parameters(), lr=1e-4)

def train_teacher(epochs, train_loader):
    teacher.train()
    for _ in range(epochs):
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            logits = teacher(x)
            loss = F.cross_entropy(logits, y)
            opt_teacher.zero_grad()
            loss.backward()
            opt_teacher.step()


class SmallCNN(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1,1))
        )
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

# Instantiate student
student = SmallCNN(num_classes=10).to(device)
opt_student = torch.optim.Adam(student.parameters(), lr=1e-4)
alpha = 0.3   # weight for CE loss
T = 4.0       # distillation temperature

def kd_loss(student_logits, teacher_logits, T=4.0):
    # Softmax with temperature
    teacher_probs = F.softmax(teacher_logits / T, dim=1)
    student_log_probs = F.log_softmax(student_logits / T, dim=1)

    # KL divergence, aggregated over batch 
    loss_kl = F.kl_div(student_log_probs, teacher_probs, reduction='batchmean')

    return (T * T) * loss_kl



# 1. 数据增强 / 预处理
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

# 2. 加载 CIFAR-10
train_dataset = datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

# 3. DataLoader
dataloader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

train_teacher(epochs=1, train_loader=dataloader)
teacher.eval()  # freeze teacher
# train_student_with_distillation
for x, y in dataloader:
    x, y = x.to(device), y.to(device)
    # Teacher forward (不需要计算梯度)
    with torch.no_grad():
        teacher_logits = teacher(x)

    # Student forward
    student_logits = student(x)

    # Hard label loss
    loss_ce = F.cross_entropy(student_logits, y)

    # Distillation loss
    loss_kd = kd_loss(student_logits, teacher_logits, T)

    # Combine
    loss = alpha * loss_ce + (1 - alpha) * loss_kd

    opt_student.zero_grad()
    loss.backward()
    opt_student.step()

    print("loss:", loss.item())


loss: 6.706850051879883
loss: 6.641338348388672
loss: 6.852497100830078
loss: 6.8808393478393555
loss: 6.776559829711914
loss: 7.481376647949219
loss: 6.0222039222717285
loss: 7.132743835449219
loss: 7.319509983062744
loss: 6.267420291900635
loss: 6.981998920440674
loss: 7.101173400878906
loss: 7.005058288574219
loss: 7.252947807312012
loss: 6.8980913162231445
loss: 7.536619663238525
loss: 6.342973232269287
loss: 6.782742500305176
loss: 6.836009979248047
loss: 7.472536087036133
loss: 7.106359958648682
loss: 6.843428134918213
loss: 6.935361862182617
loss: 6.460139274597168
loss: 6.434418201446533
loss: 7.590257167816162
loss: 6.904282569885254
loss: 7.575451850891113
loss: 6.969733238220215
loss: 7.346570014953613
loss: 6.731241703033447
loss: 6.119562149047852
loss: 6.660286903381348
loss: 6.891930103302002
loss: 5.715048789978027
loss: 6.798610687255859
loss: 7.164504528045654
loss: 6.8233866691589355
loss: 6.830433368682861
loss: 6.465048789978027
loss: 6.511832237243652
loss: 7.3898

In [18]:
# Check student performance on test set
from torchvision import datasets
from torch.utils.data import DataLoader

def evaluate(model, dataloader, device):
    model.eval()
    total_loss, correct, total = 0.0, 0, 0
    criterion = torch.nn.CrossEntropyLoss()

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            logits = model(inputs)
            loss = criterion(logits, targets)
            total_loss += loss.item() * targets.size(0)

            preds = logits.argmax(dim=1)
            correct += (preds == targets).sum().item()
            total += targets.size(0)

    avg_loss = total_loss / total
    acc = correct / total
    return {"loss": avg_loss, "accuracy": acc}

test_dataset = datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=4
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
student.to(device)
metrics = evaluate(student, test_loader, device)
print(metrics)   

{'loss': 2.1237823066711425, 'accuracy': 0.1957}


In [19]:
# Check original teacher performance on test set
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

teacher.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
metrics = evaluate(teacher, test_loader, device)
print(metrics)


{'loss': 0.2054711022377014, 'accuracy': 0.9325}


Well the student seems too low in performance. chatGPT suggested common fixes

- Train longer: more epochs/steps on CIFAR-10 (both teacher finetune and student distill). One epoch isn’t enough.
- Match capacity/aug: consider a slightly larger student (wider convs) or more aug (RandomCrop/Flip). Current 2-layer conv is very small for CIFAR-10.
- Hyperparams: try higher LR for student (e.g., 1e-3), adjust α/T (e.g., α=0.5, T=2–4), use weight decay.
- Normalize inputs: ensure CIFAR-10 mean/std normalization (e.g., mean=[0.4914,0.4822,0.4465], std=[0.2470,0.2435,0.2616]) instead of only ToTensor.
- Optimize loop: use teacher.eval() and student.train(); ensure both on the same device.

| 名字           | 模块                      | 作用     |
| ------------ | ------------------------------ | ------ |
| `teacher`    | `teacher = resnet18(...)`      | 大模型，冻结 |
| `student`    | `student = SmallCNN(...)`      | 小模型，训练 |
| `dataloader` | `dataloader = DataLoader(...)` | 提供训练样本 |
